In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [5]:
#create session in order to be capable of accessing all Spark API
spark = SparkSession \
.builder \
.appName("Intro to Spark DataFrame") \
.config("spark.some.config.option","some-value") \
.getOrCreate()



In [14]:
#define data schema for file we want to read
purchase_schema = StructType([
    StructField("Date", DateType(), True),
    StructField("Time", StringType(), True),
    StructField("City", StringType(), True),
    StructField("Item", StringType(), True),
    StructField("Total", FloatType(), True),
    StructField("Payment", FloatType(), True)
])

purchaseDataframe = spark.read.csv(
"C:/Users/aayushi srivastava/Documents/AayushiSrivastavaJobSearch/PySparkProjects/dataset/purchases.csv", header = True,
schema = purchase_schema, sep = "\t")

purchaseDataframe.show(3)

+----------+-----+----------+----------------+------+-------+
|      Date| Time|      City|            Item| Total|Payment|
+----------+-----+----------+----------------+------+-------+
|2012-01-01|09:00|  San Jose|  Men's Clothing|214.05|   null|
|2012-01-01|09:00|Fort Worth|Women's Clothing|153.57|   null|
|2012-01-01|09:00| San Diego|           Music| 66.08|   null|
+----------+-----+----------+----------------+------+-------+
only showing top 3 rows



In [15]:
#count number of rows in our dataframe
num_rows = purchaseDataframe.count()
print("Number of rows: ",num_rows)

#show our dataframe schema
purchaseDataframe.printSchema()

#show statistic of the data we want
purchaseDataframe.describe('Total').show()


Number of rows:  4138476
root
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Total: float (nullable = true)
 |-- Payment: float (nullable = true)

+-------+------------------+
|summary|             Total|
+-------+------------------+
|  count|           4138476|
|   mean|249.96108549398525|
| stddev| 144.3174111542959|
|    min|               0.0|
|    max|            499.99|
+-------+------------------+



In [17]:
#create new dataframe from "city" and "Total" columns
newDataframe = purchaseDataframe.select(purchaseDataframe['city'],purchaseDataframe['Total'])
newDataframe.show(3)
newDataframe.printSchema()

+----------+------+
|      city| Total|
+----------+------+
|  San Jose|214.05|
|Fort Worth|153.57|
| San Diego| 66.08|
+----------+------+
only showing top 3 rows

root
 |-- city: string (nullable = true)
 |-- Total: float (nullable = true)



In [18]:
#adding constant value of 10 to every row data in column 'Total'
purchaseDataframe.select(purchaseDataframe['City'],
                        purchaseDataframe['Total']+10).show(3)

+----------+------------+
|      City|(Total + 10)|
+----------+------------+
|  San Jose|      224.05|
|Fort Worth|      163.57|
| San Diego|       76.08|
+----------+------------+
only showing top 3 rows



In [19]:
#filtering dataframe
#filter only row data whose 'Total' column value > 200
purchaseDataframe.filter(purchaseDataframe['Total'] > 200).show(3)

+----------+-----+----------+-------------------+------+-------+
|      Date| Time|      City|               Item| Total|Payment|
+----------+-----+----------+-------------------+------+-------+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|   null|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|   null|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|   null|
+----------+-----+----------+-------------------+------+-------+
only showing top 3 rows



In [20]:
#sorting dataframe by certain column
sortedByCity = purchaseDataframe.orderBy('City').show(4)

+----------+-----+-----------+----------------+------+-------+
|      Date| Time|       City|            Item| Total|Payment|
+----------+-----+-----------+----------------+------+-------+
|2012-10-07|11:11|Albuquerque|    Pet Supplies| 308.7|   null|
|2012-10-07|11:40|Albuquerque|            Toys|299.63|   null|
|2012-10-07|11:13|Albuquerque|Women's Clothing|419.49|   null|
|2012-10-07|10:39|Albuquerque|    Pet Supplies| 401.3|   null|
+----------+-----+-----------+----------------+------+-------+
only showing top 4 rows



In [22]:
#calculating number of transactions in each city
numTransactionEachCity = purchaseDataframe.groupBy("City").count()
numTransactionEachCity.show(5)

+---------------+-----+
|           City|count|
+---------------+-----+
|North Las Vegas|40013|
|        Phoenix|40333|
|          Omaha|40209|
|      Anchorage|39806|
|        Anaheim|40086|
+---------------+-----+
only showing top 5 rows



In [25]:
#Indexing and Accessing DataFrame
#(create new column with inccremental id.then we can access using .filter to incremental id)

#importing monotonically increasing id
from pyspark.sql.functions import monotonically_increasing_id

newPurchasedDataframe = purchaseDataframe.withColumn(
"index", monotonically_increasing_id())
newPurchasedDataframe.show(7)

row2till4 = newPurchasedDataframe.filter((newPurchasedDataframe['index'] >= 2) &
                                        (newPurchasedDataframe['index'] <= 4))
row2till4.show()

+----------+-----+----------+-------------------+------+-------+-----+
|      Date| Time|      City|               Item| Total|Payment|index|
+----------+-----+----------+-------------------+------+-------+-----+
|2012-01-01|09:00|  San Jose|     Men's Clothing|214.05|   null|    0|
|2012-01-01|09:00|Fort Worth|   Women's Clothing|153.57|   null|    1|
|2012-01-01|09:00| San Diego|              Music| 66.08|   null|    2|
|2012-01-01|09:00|Pittsburgh|       Pet Supplies|493.51|   null|    3|
|2012-01-01|09:00|     Omaha|Children's Clothing|235.63|   null|    4|
|2012-01-01|09:00|  Stockton|     Men's Clothing|247.18|   null|    5|
|2012-01-01|09:00|    Austin|            Cameras| 379.6|   null|    6|
+----------+-----+----------+-------------------+------+-------+-----+
only showing top 7 rows

+----------+-----+----------+-------------------+------+-------+-----+
|      Date| Time|      City|               Item| Total|Payment|index|
+----------+-----+----------+-------------------+---

In [26]:
#Then, to access it by row and column use ".select()"
dataRow2ColumnTotal = newPurchasedDataframe.filter(newPurchasedDataframe['index'] == 2).select('Total')
dataRow2ColumnTotal.show()


+-----+
|Total|
+-----+
|66.08|
+-----+



In [27]:
#Using sql query in dataframe
#we need to make sql temporary view for our dataframe
purchaseDataframe.createOrReplaceTempView("purchaseSql")

#select 'Total' and "Payment" column from our sql temporary view
anotherDataframe = spark.sql("SELECT Total, Payment FROM purchasesql")
anotherDataframe.show(3)

+------+-------+
| Total|Payment|
+------+-------+
|214.05|   null|
|153.57|   null|
| 66.08|   null|
+------+-------+
only showing top 3 rows



In [28]:
#sorting data by certain column
#sorting data by city column alphabetically
orderByCity = spark.sql("SELECT * FROM purchasesql ORDER BY City")
orderByCity.show(3)

+----------+-----+-----------+-------------------+------+-------+
|      Date| Time|       City|               Item| Total|Payment|
+----------+-----+-----------+-------------------+------+-------+
|2012-10-07|10:46|Albuquerque|Children's Clothing|143.48|   null|
|2012-10-07|11:18|Albuquerque|             Crafts|475.77|   null|
|2012-10-07|11:11|Albuquerque|       Pet Supplies| 308.7|   null|
+----------+-----+-----------+-------------------+------+-------+
only showing top 3 rows



In [29]:
#filter data with 'Total' column value > 200 and sort them by Payment column data
filterAndSortWithSQL = spark.sql("SELECT * FROM purchasesql WHERE Total > 200 ORDER BY payment")
filterAndSortWithSQL.show(4)

+----------+-----+-----------+--------------------+------+-------+
|      Date| Time|       City|                Item| Total|Payment|
+----------+-----+-----------+--------------------+------+-------+
|2012-10-07|10:32|     Toledo|               Music|414.82|   null|
|2012-10-07|10:32| Chesapeake|      Sporting Goods|377.16|   null|
|2012-10-07|10:32|   San Jose|Consumer Electronics|234.27|   null|
|2012-10-07|10:32|Bakersfield|   Health and Beauty| 375.2|   null|
+----------+-----+-----------+--------------------+------+-------+
only showing top 4 rows

